In [1]:
import os
import pandas as pd

In [2]:
print(os.listdir('../input/glass-qualitymachine-hack'))

['Sample_Submission.xlsx', 'Test.csv', 'Train.csv']


In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

plt.rcParams['figure.figsize'] = [9, 12]

import warnings
warnings.simplefilter('ignore')

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score, train_test_split, KFold, StratifiedKFold

# Classification
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier,ExtraTreesClassifier,VotingClassifier
from catboost import CatBoostClassifier

import lightgbm as lgb
import xgboost as xgb

In [22]:
train_1 = pd.read_csv("../input/glass-qualitymachine-hack/Train.csv")
test_1 = pd.read_csv("../input/glass-qualitymachine-hack/Test.csv")

In [23]:
train = train_1.copy()
test = test_1.copy()

In [24]:
train.head()

,grade_A_Component_1,grade_A_Component_2,max_luminosity,thickness,xmin,xmax,ymin,ymax,pixel_area,log_area,x_component_1,x_component_2,x_component_3,x_component_4,x_component_5,class
0,0,1,150,36,144,172,947225,947332,439,439.099273,0,0,1,0,0,1
1,1,0,134,55,1144,1152,2379058,2379624,329,329.205616,1,0,0,0,0,1
2,1,0,135,65,950,974,1038442,1036754,300,300.120598,0,0,0,0,0,2
3,0,1,123,35,41,220,1705580,1705604,6803,6803.778622,0,0,1,0,0,1
4,1,0,138,69,462,466,1088124,1086579,251,251.401943,0,0,0,0,0,2


In [25]:
train.head()

,grade_A_Component_1,grade_A_Component_2,max_luminosity,thickness,xmin,xmax,ymin,ymax,pixel_area,log_area,x_component_1,x_component_2,x_component_3,x_component_4,x_component_5,class
0,0,1,150,36,144,172,947225,947332,439,439.099273,0,0,1,0,0,1
1,1,0,134,55,1144,1152,2379058,2379624,329,329.205616,1,0,0,0,0,1
2,1,0,135,65,950,974,1038442,1036754,300,300.120598,0,0,0,0,0,2
3,0,1,123,35,41,220,1705580,1705604,6803,6803.778622,0,0,1,0,0,1
4,1,0,138,69,462,466,1088124,1086579,251,251.401943,0,0,0,0,0,2


In [26]:
def weight(Xmin,Xmax,Ymin,Ymax,thickness):
    Xmin = Xmin/100
    Xmax = Xmax/100
    Ymin = Ymin/100
    Ymax = Ymax/100
    
    length = Xmax-Xmin
    width = Ymax-Ymin
    ans = length*width*thickness*2.5
    return ans

In [27]:
result=[]
for i in range(len(train)):
    ans = weight(train['xmin'][i], train['xmax'][i], train['ymin'][i], train['ymax'][i], train['thickness'][i])
    result.append(ans)

In [28]:
train['class'].value_counts()

1    887
2    471
Name: class, dtype: int64

In [29]:
train['sum_x_component_1'] = train['x_component_1']+train['x_component_2']+train['x_component_3']+train['x_component_4']+train['x_component_5']
test['sum_x_component_1'] = test['x_component_1']+test['x_component_2']+test['x_component_3']+test['x_component_4']+test['x_component_5']

In [30]:
train.columns

Index(['grade_A_Component_1', 'grade_A_Component_2', 'max_luminosity',
       'thickness', 'xmin', 'xmax', 'ymin', 'ymax', 'pixel_area', 'log_area',
       'x_component_1', 'x_component_2', 'x_component_3', 'x_component_4',
       'x_component_5', 'class', 'sum_x_component_1'],
      dtype='object')

In [31]:
target = 'class'
feat = train.columns.to_list()
delete = [target]
for i in delete:
    feat.remove(i)

In [32]:
train.corr()

,grade_A_Component_1,grade_A_Component_2,max_luminosity,thickness,xmin,xmax,ymin,ymax,pixel_area,log_area,x_component_1,x_component_2,x_component_3,x_component_4,x_component_5,class,sum_x_component_1
grade_A_Component_1,1.000000,-1.000000,-0.179342,0.102448,0.122591,0.090869,0.075011,0.074997,-0.288747,-0.288750,-0.060848,0.357737,-0.410352,-0.150293,-0.085870,-0.017173,-0.224719
grade_A_Component_2,-1.000000,1.000000,0.179342,-0.102448,-0.122591,-0.090869,-0.075011,-0.074997,0.288747,0.288750,0.060848,-0.357737,0.410352,0.150293,0.085870,0.017173,0.224719
max_luminosity,-0.179342,0.179342,1.000000,-0.084196,-0.064511,-0.052570,-0.047701,-0.047675,0.093665,0.093664,-0.050434,-0.194116,0.150255,0.071122,0.018768,0.031459,0.004920
thickness,0.102448,-0.102448,-0.084196,1.000000,0.132322,0.102853,-0.215164,-0.215179,-0.222397,-0.222396,0.067869,-0.027194,-0.345223,-0.098207,0.016376,0.329639,-0.286749
xmin,0.122591,-0.122591,-0.064511,0.132322,1.000000,0.986944,0.052343,0.052325,-0.373870,-0.373869,0.131579,-0.237880,-0.397686,0.071128,0.110482,0.147477,-0.329989
xmax,0.090869,-0.090869,-0.052570,0.102853,0.986944,1.000000,0.065922,0.065907,-0.264107,-0.264106,0.115860,-0.267536,-0.315293,0.058899,0.103368,0.130023,-0.297971
ymin,0.075011,-0.075011,-0.047701,-0.215164,0.052343,0.065922,1.000000,1.000000,0.006516,0.006514,0.016542,-0.071391,0.018013,-0.062412,0.051208,-0.092489,-0.026860
ymax,0.074997,-0.074997,-0.047675,-0.215179,0.052325,0.065907,1.000000,1.000000,0.006556,0.006553,0.016565,-0.071419,0.018027,-0.062421,0.051208,-0.092485,-0.026857
pixel_area,-0.288747,0.288747,0.093665,-0.222397,-0.373870,-0.264107,0.006516,0.006556,1.000000,1.000000,-0.098369,-0.112656,0.677638,-0.086455,-0.064310,-0.214127,0.364931
log_area,-0.288750,0.288750,0.093664,-0.222396,-0.373869,-0.264106,0.006514,0.006553,1.000000,1.000000,-0.098368,-0.112655,0.677639,-0.086454,-0.064311,-0.214127,0.364934


## Smote

In [33]:
# from imblearn.over_sampling import SMOTE
# sm = SMOTE(random_state=13,sampling_strategy='all')
# X_train_ovr,y_train_ovr = sm.fit_sample(train[feat],train[target])
# print(X_train_ovr.shape)

In [34]:
X_train_ovr.duplicated().sum()

0

In [48]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import log_loss,make_scorer
def baseliner(train, features, target, cv=10, metric='accuracy'):
    """
    Function for baselining Models which return CV Score, Train Score, Valid Score
    """
    print("Baseliner Models\n")
    eval_dict = {}
    random_state = 2
    models = [lgb.LGBMClassifier(), xgb.XGBClassifier(), GradientBoostingClassifier(), LogisticRegression(),CatBoostClassifier(depth=8,iterations=253),
              RandomForestClassifier(random_state=random_state), DecisionTreeClassifier(), AdaBoostClassifier(),ExtraTreesClassifier(),MLPClassifier(hidden_layer_sizes=250,random_state=42)
             ]
    print("Model Name \t |   CV") #    | \t TRN   | \t  VAL
    print("--" * 50)

    for index, model in enumerate(models, 0):
        model_name = str(model).split("(")[0]
        eval_dict[model_name] = {}
        
        Log_loss = make_scorer(log_loss,greater_is_better=False, needs_proba=True)
        results = cross_val_score(model, train[features], train[target], cv=10, scoring=Log_loss)
        eval_dict[model_name]['log_loss'] = results.mean()

        print("%s \t | %.4f \t" % (
            model_name[:12], eval_dict[model_name]['log_loss']))


In [49]:
X_train_ovr['class']= y_train_ovr

In [50]:
X_train_ovr[feat].head()

,grade_A_Component_1,grade_A_Component_2,max_luminosity,thickness,xmin,xmax,ymin,ymax,pixel_area,log_area,x_component_1,x_component_2,x_component_3,x_component_4,x_component_5,sum_x_component_1
0,0,1,150,36,144,172,947225,947332,439,439.099273,0,0,1,0,0,1
1,1,0,134,55,1144,1152,2379058,2379624,329,329.205616,1,0,0,0,0,1
2,1,0,135,65,950,974,1038442,1036754,300,300.120598,0,0,0,0,0,0
3,0,1,123,35,41,220,1705580,1705604,6803,6803.778622,0,0,1,0,0,1
4,1,0,138,69,462,466,1088124,1086579,251,251.401943,0,0,0,0,0,0


In [51]:
baseliner(X_train_ovr, feat, target)

Baseliner Models

Model Name 	 |   CV
----------------------------------------------------------------------------------------------------
LGBMClassifi 	 | -0.2709 	
XGBClassifie 	 | -0.2765 	
GradientBoos 	 | -0.2635 	
LogisticRegr 	 | -0.6453 	
Learning rate set to 0.044358
0:	learn: 0.6421741	total: 7.79ms	remaining: 1.96s
1:	learn: 0.5968034	total: 13.9ms	remaining: 1.75s
2:	learn: 0.5578230	total: 20ms	remaining: 1.67s
3:	learn: 0.5233733	total: 24.6ms	remaining: 1.53s
4:	learn: 0.4926956	total: 26.8ms	remaining: 1.33s
5:	learn: 0.4654329	total: 33.1ms	remaining: 1.36s
6:	learn: 0.4412710	total: 39.5ms	remaining: 1.39s
7:	learn: 0.4220973	total: 45.7ms	remaining: 1.4s
8:	learn: 0.4037976	total: 51.7ms	remaining: 1.4s
9:	learn: 0.3919540	total: 57.9ms	remaining: 1.41s
10:	learn: 0.3773644	total: 64ms	remaining: 1.41s
11:	learn: 0.3662837	total: 70.2ms	remaining: 1.41s
12:	learn: 0.3546299	total: 76.4ms	remaining: 1.41s
13:	learn: 0.3450406	total: 82.5ms	remaining: 1.41s
14:	learn: 

188:	learn: 0.1118658	total: 971ms	remaining: 329ms
189:	learn: 0.1111526	total: 976ms	remaining: 324ms
190:	learn: 0.1109440	total: 981ms	remaining: 318ms
191:	learn: 0.1101425	total: 986ms	remaining: 313ms
192:	learn: 0.1095878	total: 991ms	remaining: 308ms
193:	learn: 0.1090972	total: 996ms	remaining: 303ms
194:	learn: 0.1088694	total: 1s	remaining: 298ms
195:	learn: 0.1080979	total: 1.01s	remaining: 293ms
196:	learn: 0.1075295	total: 1.01s	remaining: 287ms
197:	learn: 0.1069632	total: 1.02s	remaining: 282ms
198:	learn: 0.1063872	total: 1.02s	remaining: 277ms
199:	learn: 0.1060861	total: 1.03s	remaining: 272ms
200:	learn: 0.1056542	total: 1.03s	remaining: 267ms
201:	learn: 0.1053548	total: 1.04s	remaining: 262ms
202:	learn: 0.1048636	total: 1.04s	remaining: 257ms
203:	learn: 0.1043942	total: 1.05s	remaining: 252ms
204:	learn: 0.1041238	total: 1.05s	remaining: 247ms
205:	learn: 0.1037557	total: 1.06s	remaining: 241ms
206:	learn: 0.1031665	total: 1.06s	remaining: 236ms
207:	learn: 0.1

121:	learn: 0.1398284	total: 608ms	remaining: 653ms
122:	learn: 0.1392192	total: 613ms	remaining: 648ms
123:	learn: 0.1387513	total: 618ms	remaining: 643ms
124:	learn: 0.1383018	total: 624ms	remaining: 639ms
125:	learn: 0.1377863	total: 628ms	remaining: 633ms
126:	learn: 0.1369741	total: 633ms	remaining: 628ms
127:	learn: 0.1363894	total: 638ms	remaining: 623ms
128:	learn: 0.1355897	total: 643ms	remaining: 618ms
129:	learn: 0.1351077	total: 648ms	remaining: 613ms
130:	learn: 0.1346035	total: 653ms	remaining: 608ms
131:	learn: 0.1335263	total: 658ms	remaining: 603ms
132:	learn: 0.1329205	total: 663ms	remaining: 599ms
133:	learn: 0.1325600	total: 669ms	remaining: 594ms
134:	learn: 0.1322378	total: 674ms	remaining: 589ms
135:	learn: 0.1318767	total: 680ms	remaining: 585ms
136:	learn: 0.1312569	total: 685ms	remaining: 580ms
137:	learn: 0.1305556	total: 690ms	remaining: 575ms
138:	learn: 0.1300825	total: 695ms	remaining: 570ms
139:	learn: 0.1295644	total: 700ms	remaining: 565ms
140:	learn: 

64:	learn: 0.2063546	total: 297ms	remaining: 860ms
65:	learn: 0.2045861	total: 302ms	remaining: 856ms
66:	learn: 0.2032259	total: 307ms	remaining: 852ms
67:	learn: 0.2022781	total: 311ms	remaining: 847ms
68:	learn: 0.2009618	total: 316ms	remaining: 843ms
69:	learn: 0.1995449	total: 321ms	remaining: 838ms
70:	learn: 0.1989281	total: 325ms	remaining: 834ms
71:	learn: 0.1975208	total: 330ms	remaining: 829ms
72:	learn: 0.1961000	total: 334ms	remaining: 825ms
73:	learn: 0.1951201	total: 339ms	remaining: 820ms
74:	learn: 0.1931908	total: 344ms	remaining: 815ms
75:	learn: 0.1928493	total: 345ms	remaining: 804ms
76:	learn: 0.1914089	total: 350ms	remaining: 800ms
77:	learn: 0.1907065	total: 355ms	remaining: 796ms
78:	learn: 0.1896541	total: 360ms	remaining: 792ms
79:	learn: 0.1885796	total: 365ms	remaining: 788ms
80:	learn: 0.1878239	total: 369ms	remaining: 784ms
81:	learn: 0.1870939	total: 374ms	remaining: 780ms
82:	learn: 0.1860392	total: 379ms	remaining: 776ms
83:	learn: 0.1849764	total: 383

229:	learn: 0.0908214	total: 1.08s	remaining: 109ms
230:	learn: 0.0904870	total: 1.09s	remaining: 104ms
231:	learn: 0.0901849	total: 1.09s	remaining: 99.1ms
232:	learn: 0.0897604	total: 1.1s	remaining: 94.4ms
233:	learn: 0.0890985	total: 1.1s	remaining: 89.7ms
234:	learn: 0.0890290	total: 1.11s	remaining: 85ms
235:	learn: 0.0884531	total: 1.11s	remaining: 80.3ms
236:	learn: 0.0879890	total: 1.12s	remaining: 75.6ms
237:	learn: 0.0876457	total: 1.12s	remaining: 70.9ms
238:	learn: 0.0871370	total: 1.13s	remaining: 66.2ms
239:	learn: 0.0869085	total: 1.13s	remaining: 61.4ms
240:	learn: 0.0864988	total: 1.14s	remaining: 56.7ms
241:	learn: 0.0860867	total: 1.14s	remaining: 52ms
242:	learn: 0.0857200	total: 1.15s	remaining: 47.3ms
243:	learn: 0.0854433	total: 1.15s	remaining: 42.5ms
244:	learn: 0.0850935	total: 1.16s	remaining: 37.8ms
245:	learn: 0.0845692	total: 1.16s	remaining: 33.1ms
246:	learn: 0.0841720	total: 1.17s	remaining: 28.4ms
247:	learn: 0.0837534	total: 1.17s	remaining: 23.6ms
2

...

140:	learn: 0.1401066	total: 663ms	remaining: 527ms
141:	learn: 0.1397378	total: 668ms	remaining: 522ms
142:	learn: 0.1393192	total: 673ms	remaining: 518ms
143:	learn: 0.1387331	total: 678ms	remaining: 513ms
144:	learn: 0.1383950	total: 683ms	remaining: 508ms
145:	learn: 0.1379366	total: 687ms	remaining: 504ms
146:	learn: 0.1368459	total: 692ms	remaining: 499ms
147:	learn: 0.1360055	total: 696ms	remaining: 494ms
148:	learn: 0.1353692	total: 701ms	remaining: 489ms
149:	learn: 0.1341364	total: 706ms	remaining: 485ms
150:	learn: 0.1335109	total: 711ms	remaining: 480ms
151:	learn: 0.1327840	total: 715ms	remaining: 475ms
152:	learn: 0.1323267	total: 720ms	remaining: 471ms
153:	learn: 0.1318475	total: 725ms	remaining: 466ms
154:	learn: 0.1313965	total: 729ms	remaining: 461ms
155:	learn: 0.1307843	total: 734ms	remaining: 457ms
156:	learn: 0.1302186	total: 739ms	remaining: 452ms
157:	learn: 0.1296756	total: 744ms	remaining: 447ms
158:	learn: 0.1291071	total: 748ms	remaining: 442ms
159:	learn: 

85:	learn: 0.1848349	total: 404ms	remaining: 785ms
86:	learn: 0.1831892	total: 410ms	remaining: 783ms
87:	learn: 0.1829028	total: 415ms	remaining: 779ms
88:	learn: 0.1823536	total: 420ms	remaining: 775ms
89:	learn: 0.1810153	total: 426ms	remaining: 771ms
90:	learn: 0.1801405	total: 431ms	remaining: 767ms
91:	learn: 0.1796591	total: 435ms	remaining: 762ms
92:	learn: 0.1786283	total: 440ms	remaining: 757ms
93:	learn: 0.1776066	total: 445ms	remaining: 753ms
94:	learn: 0.1766291	total: 449ms	remaining: 747ms
95:	learn: 0.1752683	total: 454ms	remaining: 743ms
96:	learn: 0.1746781	total: 459ms	remaining: 738ms
97:	learn: 0.1742818	total: 461ms	remaining: 730ms
98:	learn: 0.1736535	total: 466ms	remaining: 725ms
99:	learn: 0.1730776	total: 471ms	remaining: 721ms
100:	learn: 0.1723407	total: 476ms	remaining: 716ms
101:	learn: 0.1714979	total: 481ms	remaining: 712ms
102:	learn: 0.1703170	total: 485ms	remaining: 707ms
103:	learn: 0.1699456	total: 487ms	remaining: 698ms
104:	learn: 0.1692976	total

13:	learn: 0.3514594	total: 65.9ms	remaining: 1.12s
14:	learn: 0.3429845	total: 72.6ms	remaining: 1.15s
15:	learn: 0.3348785	total: 77.5ms	remaining: 1.15s
16:	learn: 0.3285360	total: 78.9ms	remaining: 1.09s
17:	learn: 0.3215468	total: 83.8ms	remaining: 1.09s
18:	learn: 0.3144842	total: 88.4ms	remaining: 1.09s
19:	learn: 0.3085364	total: 93.2ms	remaining: 1.09s
20:	learn: 0.3035339	total: 98.2ms	remaining: 1.08s
21:	learn: 0.2982905	total: 103ms	remaining: 1.08s
22:	learn: 0.2943966	total: 108ms	remaining: 1.08s
23:	learn: 0.2900770	total: 112ms	remaining: 1.07s
24:	learn: 0.2869651	total: 118ms	remaining: 1.08s
25:	learn: 0.2827053	total: 124ms	remaining: 1.08s
26:	learn: 0.2798063	total: 130ms	remaining: 1.09s
27:	learn: 0.2775934	total: 136ms	remaining: 1.09s
28:	learn: 0.2754054	total: 142ms	remaining: 1.1s
29:	learn: 0.2719511	total: 148ms	remaining: 1.1s
30:	learn: 0.2692203	total: 154ms	remaining: 1.1s
31:	learn: 0.2659317	total: 160ms	remaining: 1.11s
32:	learn: 0.2631219	total

210:	learn: 0.1058207	total: 1.05s	remaining: 209ms
211:	learn: 0.1052259	total: 1.05s	remaining: 204ms
212:	learn: 0.1045030	total: 1.06s	remaining: 199ms
213:	learn: 0.1040923	total: 1.06s	remaining: 194ms
214:	learn: 0.1036317	total: 1.07s	remaining: 189ms
215:	learn: 0.1031344	total: 1.07s	remaining: 184ms
216:	learn: 0.1026231	total: 1.08s	remaining: 179ms
217:	learn: 0.1020819	total: 1.08s	remaining: 174ms
218:	learn: 0.1018351	total: 1.09s	remaining: 169ms
219:	learn: 0.1015262	total: 1.09s	remaining: 164ms
220:	learn: 0.1009828	total: 1.1s	remaining: 159ms
221:	learn: 0.1004814	total: 1.1s	remaining: 154ms
222:	learn: 0.0999980	total: 1.1s	remaining: 149ms
223:	learn: 0.0994363	total: 1.11s	remaining: 144ms
224:	learn: 0.0990418	total: 1.11s	remaining: 139ms
225:	learn: 0.0985546	total: 1.12s	remaining: 134ms
226:	learn: 0.0980851	total: 1.13s	remaining: 129ms
227:	learn: 0.0976258	total: 1.13s	remaining: 124ms
228:	learn: 0.0974400	total: 1.13s	remaining: 119ms
229:	learn: 0.0

157:	learn: 0.1345248	total: 730ms	remaining: 439ms
158:	learn: 0.1342223	total: 735ms	remaining: 435ms
159:	learn: 0.1340217	total: 740ms	remaining: 430ms
160:	learn: 0.1334871	total: 744ms	remaining: 425ms
161:	learn: 0.1326540	total: 749ms	remaining: 421ms
162:	learn: 0.1320145	total: 754ms	remaining: 416ms
163:	learn: 0.1312663	total: 758ms	remaining: 411ms
164:	learn: 0.1307294	total: 763ms	remaining: 407ms
165:	learn: 0.1299430	total: 767ms	remaining: 402ms
166:	learn: 0.1294735	total: 772ms	remaining: 397ms
167:	learn: 0.1289856	total: 776ms	remaining: 393ms
168:	learn: 0.1280595	total: 781ms	remaining: 388ms
169:	learn: 0.1273882	total: 785ms	remaining: 383ms
170:	learn: 0.1266761	total: 790ms	remaining: 379ms
171:	learn: 0.1262612	total: 795ms	remaining: 374ms
172:	learn: 0.1256766	total: 799ms	remaining: 370ms
173:	learn: 0.1249319	total: 804ms	remaining: 365ms
174:	learn: 0.1243509	total: 808ms	remaining: 360ms
175:	learn: 0.1238732	total: 813ms	remaining: 356ms
176:	learn: 

## Voting Classifier

In [52]:
len(feat)

16

In [53]:
from sklearn.metrics import make_scorer,log_loss
ExtC = ExtraTreesClassifier()
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve
kfold = StratifiedKFold(n_splits=10)

## Search grid for optimal parameters
ex_param_grid = {"max_depth": [None],
                 "max_features":[3,6,7,11],
              "min_samples_split": [2],
              "min_samples_leaf": [1],
              "bootstrap": [False],
              "n_estimators" :[100],
              "criterion": ["gini"]}

LogLoss = make_scorer(log_loss, greater_is_better=False, needs_proba=True)

gsExtC = GridSearchCV(ExtC,param_grid = ex_param_grid, cv=kfold, scoring=LogLoss, n_jobs= -1, verbose = 1)

gsExtC.fit(X_train_ovr[feat],X_train_ovr[target])

ExtC_best = gsExtC.best_estimator_

# Best score
gsExtC.best_score_


Fitting 10 folds for each of 4 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    4.9s finished


-0.21295486746186745

In [54]:
ExtC_best

ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features=11,
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=100,
                     n_jobs=None, oob_score=False, random_state=None, verbose=0,
                     warm_start=False)

In [55]:
LogLoss = make_scorer(log_loss, greater_is_better=False, needs_proba=True)
result=cross_val_score(gsExtC,X_train_ovr[feat],X_train_ovr[target], cv=10, scoring=LogLoss)

Fitting 10 folds for each of 4 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    3.3s finished


Fitting 10 folds for each of 4 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    3.2s finished


Fitting 10 folds for each of 4 candidates, totalling 40 fits


...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    3.3s finished


Fitting 10 folds for each of 4 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    3.3s finished


Fitting 10 folds for each of 4 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    3.4s finished


In [56]:
result.mean()

-0.23094458184725655

In [57]:
ExtC_best

ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features=11,
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=100,
                     n_jobs=None, oob_score=False, random_state=None, verbose=0,
                     warm_start=False)

In [58]:
clf = CatBoostClassifier(depth=8,iterations=253)
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X_train_ovr[feat],X_train_ovr[target],random_state=13,test_size=.1,stratify=X_train_ovr[target].values)
clf.fit(X_train_ovr[feat],X_train_ovr[target])

Learning rate set to 0.046406
0:	learn: 0.6387899	total: 6.62ms	remaining: 1.67s
1:	learn: 0.5908456	total: 11.6ms	remaining: 1.46s
2:	learn: 0.5508750	total: 16.3ms	remaining: 1.36s
3:	learn: 0.5151704	total: 19.8ms	remaining: 1.23s
4:	learn: 0.4835640	total: 21.7ms	remaining: 1.08s
5:	learn: 0.4558353	total: 26.7ms	remaining: 1.1s
6:	learn: 0.4314764	total: 31.5ms	remaining: 1.1s
7:	learn: 0.4132165	total: 36.2ms	remaining: 1.11s
8:	learn: 0.3954638	total: 41ms	remaining: 1.11s
9:	learn: 0.3830831	total: 46.5ms	remaining: 1.13s
10:	learn: 0.3701667	total: 51.6ms	remaining: 1.14s
11:	learn: 0.3573822	total: 56.3ms	remaining: 1.13s
12:	learn: 0.3461659	total: 61.5ms	remaining: 1.13s
13:	learn: 0.3366500	total: 66.3ms	remaining: 1.13s
14:	learn: 0.3287339	total: 71.4ms	remaining: 1.13s
15:	learn: 0.3219549	total: 76.5ms	remaining: 1.13s
16:	learn: 0.3161429	total: 77.9ms	remaining: 1.08s
17:	learn: 0.3095177	total: 83.2ms	remaining: 1.09s
18:	learn: 0.3026226	total: 88.1ms	remaining: 1.

195:	learn: 0.1052907	total: 980ms	remaining: 285ms
196:	learn: 0.1050516	total: 985ms	remaining: 280ms
197:	learn: 0.1046245	total: 990ms	remaining: 275ms
198:	learn: 0.1043411	total: 994ms	remaining: 270ms
199:	learn: 0.1039696	total: 1000ms	remaining: 265ms
200:	learn: 0.1033084	total: 1s	remaining: 260ms
201:	learn: 0.1029085	total: 1.01s	remaining: 255ms
202:	learn: 0.1022631	total: 1.01s	remaining: 250ms
203:	learn: 0.1018805	total: 1.02s	remaining: 245ms
204:	learn: 0.1013485	total: 1.02s	remaining: 240ms
205:	learn: 0.1010713	total: 1.03s	remaining: 235ms
206:	learn: 0.1005506	total: 1.03s	remaining: 230ms
207:	learn: 0.1001099	total: 1.04s	remaining: 225ms
208:	learn: 0.0996885	total: 1.04s	remaining: 220ms
209:	learn: 0.0992871	total: 1.05s	remaining: 214ms
210:	learn: 0.0988091	total: 1.05s	remaining: 209ms
211:	learn: 0.0983742	total: 1.06s	remaining: 204ms
212:	learn: 0.0980937	total: 1.06s	remaining: 199ms
213:	learn: 0.0976205	total: 1.07s	remaining: 194ms
214:	learn: 0.

In [59]:
LogLoss = make_scorer(log_loss, greater_is_better=False, needs_proba=True)
result=cross_val_score(clf,X_train_ovr[feat],X_train_ovr[target], cv=10, scoring=LogLoss)

Learning rate set to 0.044358
0:	learn: 0.6421741	total: 5.36ms	remaining: 1.35s
1:	learn: 0.5968034	total: 10.1ms	remaining: 1.26s
2:	learn: 0.5578230	total: 14.8ms	remaining: 1.23s
3:	learn: 0.5233733	total: 18.3ms	remaining: 1.14s
4:	learn: 0.4926956	total: 19.9ms	remaining: 985ms
5:	learn: 0.4654329	total: 24.4ms	remaining: 1.01s
6:	learn: 0.4412710	total: 29.2ms	remaining: 1.02s
7:	learn: 0.4220973	total: 33.7ms	remaining: 1.03s
8:	learn: 0.4037976	total: 38.7ms	remaining: 1.05s
9:	learn: 0.3919540	total: 43.8ms	remaining: 1.06s
10:	learn: 0.3773644	total: 48.6ms	remaining: 1.07s
11:	learn: 0.3662837	total: 53.7ms	remaining: 1.08s
12:	learn: 0.3546299	total: 58.8ms	remaining: 1.08s
13:	learn: 0.3450406	total: 63.5ms	remaining: 1.08s
14:	learn: 0.3366027	total: 68.3ms	remaining: 1.08s
15:	learn: 0.3291364	total: 73.2ms	remaining: 1.08s
16:	learn: 0.3228540	total: 74.6ms	remaining: 1.03s
17:	learn: 0.3162370	total: 79.7ms	remaining: 1.04s
18:	learn: 0.3104827	total: 84.5ms	remaining

159:	learn: 0.1279632	total: 778ms	remaining: 452ms
160:	learn: 0.1275007	total: 783ms	remaining: 447ms
161:	learn: 0.1270362	total: 787ms	remaining: 442ms
162:	learn: 0.1264701	total: 792ms	remaining: 437ms
163:	learn: 0.1255919	total: 796ms	remaining: 432ms
164:	learn: 0.1250783	total: 801ms	remaining: 427ms
165:	learn: 0.1243758	total: 805ms	remaining: 422ms
166:	learn: 0.1237593	total: 810ms	remaining: 417ms
167:	learn: 0.1233814	total: 815ms	remaining: 412ms
168:	learn: 0.1229697	total: 820ms	remaining: 407ms
169:	learn: 0.1224903	total: 824ms	remaining: 402ms
170:	learn: 0.1218886	total: 829ms	remaining: 398ms
171:	learn: 0.1216448	total: 834ms	remaining: 393ms
172:	learn: 0.1211584	total: 838ms	remaining: 388ms
173:	learn: 0.1206005	total: 843ms	remaining: 383ms
174:	learn: 0.1201627	total: 847ms	remaining: 378ms
175:	learn: 0.1195782	total: 852ms	remaining: 373ms
176:	learn: 0.1191036	total: 857ms	remaining: 368ms
177:	learn: 0.1185206	total: 861ms	remaining: 363ms
178:	learn: 

65:	learn: 0.1878943	total: 299ms	remaining: 847ms
66:	learn: 0.1867084	total: 304ms	remaining: 845ms
67:	learn: 0.1854697	total: 309ms	remaining: 840ms
68:	learn: 0.1847521	total: 314ms	remaining: 837ms
69:	learn: 0.1838952	total: 318ms	remaining: 832ms
70:	learn: 0.1825321	total: 323ms	remaining: 828ms
71:	learn: 0.1809807	total: 327ms	remaining: 823ms
72:	learn: 0.1802273	total: 332ms	remaining: 819ms
73:	learn: 0.1788942	total: 337ms	remaining: 815ms
74:	learn: 0.1781450	total: 341ms	remaining: 810ms
75:	learn: 0.1776109	total: 346ms	remaining: 806ms
76:	learn: 0.1765044	total: 351ms	remaining: 802ms
77:	learn: 0.1755026	total: 355ms	remaining: 798ms
78:	learn: 0.1746757	total: 360ms	remaining: 793ms
79:	learn: 0.1733697	total: 365ms	remaining: 788ms
80:	learn: 0.1725515	total: 369ms	remaining: 784ms
81:	learn: 0.1715807	total: 374ms	remaining: 780ms
82:	learn: 0.1708873	total: 379ms	remaining: 776ms
83:	learn: 0.1703451	total: 383ms	remaining: 771ms
84:	learn: 0.1700935	total: 385

235:	learn: 0.0823309	total: 1.09s	remaining: 78.4ms
236:	learn: 0.0819642	total: 1.09s	remaining: 73.8ms
237:	learn: 0.0816734	total: 1.1s	remaining: 69.2ms
238:	learn: 0.0815240	total: 1.1s	remaining: 64.6ms
239:	learn: 0.0810544	total: 1.11s	remaining: 60ms
240:	learn: 0.0806740	total: 1.11s	remaining: 55.3ms
241:	learn: 0.0801652	total: 1.12s	remaining: 50.7ms
242:	learn: 0.0798058	total: 1.12s	remaining: 46.1ms
243:	learn: 0.0795529	total: 1.13s	remaining: 41.5ms
244:	learn: 0.0793003	total: 1.13s	remaining: 36.9ms
245:	learn: 0.0790479	total: 1.13s	remaining: 32.3ms
246:	learn: 0.0787374	total: 1.14s	remaining: 27.7ms
247:	learn: 0.0783064	total: 1.14s	remaining: 23.1ms
248:	learn: 0.0780564	total: 1.15s	remaining: 18.4ms
249:	learn: 0.0777050	total: 1.15s	remaining: 13.8ms
250:	learn: 0.0774423	total: 1.16s	remaining: 9.22ms
251:	learn: 0.0771640	total: 1.16s	remaining: 4.61ms
252:	learn: 0.0769894	total: 1.17s	remaining: 0us
Learning rate set to 0.044358
0:	learn: 0.6419287	tot

142:	learn: 0.1376117	total: 657ms	remaining: 505ms
143:	learn: 0.1373073	total: 663ms	remaining: 502ms
144:	learn: 0.1364889	total: 668ms	remaining: 497ms
145:	learn: 0.1358949	total: 673ms	remaining: 493ms
146:	learn: 0.1350301	total: 678ms	remaining: 489ms
147:	learn: 0.1346920	total: 683ms	remaining: 484ms
148:	learn: 0.1340959	total: 687ms	remaining: 480ms
149:	learn: 0.1333621	total: 693ms	remaining: 476ms
150:	learn: 0.1322164	total: 698ms	remaining: 471ms
151:	learn: 0.1315349	total: 702ms	remaining: 467ms
152:	learn: 0.1309914	total: 707ms	remaining: 462ms
153:	learn: 0.1303600	total: 712ms	remaining: 458ms
154:	learn: 0.1295038	total: 717ms	remaining: 453ms
155:	learn: 0.1285837	total: 721ms	remaining: 448ms
156:	learn: 0.1281236	total: 726ms	remaining: 444ms
157:	learn: 0.1271437	total: 731ms	remaining: 439ms
158:	learn: 0.1266824	total: 736ms	remaining: 435ms
159:	learn: 0.1260459	total: 742ms	remaining: 432ms
160:	learn: 0.1255601	total: 748ms	remaining: 427ms
161:	learn: 

...

213:	learn: 0.0992884	total: 984ms	remaining: 179ms
214:	learn: 0.0988598	total: 989ms	remaining: 175ms
215:	learn: 0.0985713	total: 993ms	remaining: 170ms
216:	learn: 0.0980866	total: 998ms	remaining: 166ms
217:	learn: 0.0973908	total: 1s	remaining: 161ms
218:	learn: 0.0971287	total: 1.01s	remaining: 156ms
219:	learn: 0.0968145	total: 1.01s	remaining: 152ms
220:	learn: 0.0964382	total: 1.02s	remaining: 147ms
221:	learn: 0.0958104	total: 1.02s	remaining: 143ms
222:	learn: 0.0954096	total: 1.02s	remaining: 138ms
223:	learn: 0.0947505	total: 1.03s	remaining: 133ms
224:	learn: 0.0943403	total: 1.03s	remaining: 129ms
225:	learn: 0.0938392	total: 1.04s	remaining: 124ms
226:	learn: 0.0934074	total: 1.04s	remaining: 120ms
227:	learn: 0.0929592	total: 1.05s	remaining: 115ms
228:	learn: 0.0927815	total: 1.05s	remaining: 110ms
229:	learn: 0.0923897	total: 1.06s	remaining: 106ms
230:	learn: 0.0920248	total: 1.06s	remaining: 101ms
231:	learn: 0.0916816	total: 1.07s	remaining: 96.7ms
232:	learn: 0.

126:	learn: 0.1503230	total: 589ms	remaining: 585ms
127:	learn: 0.1497327	total: 594ms	remaining: 580ms
128:	learn: 0.1486682	total: 600ms	remaining: 577ms
129:	learn: 0.1481684	total: 604ms	remaining: 572ms
130:	learn: 0.1472980	total: 609ms	remaining: 567ms
131:	learn: 0.1468260	total: 614ms	remaining: 563ms
132:	learn: 0.1465711	total: 616ms	remaining: 556ms
133:	learn: 0.1452029	total: 620ms	remaining: 551ms
134:	learn: 0.1450254	total: 625ms	remaining: 546ms
135:	learn: 0.1444842	total: 630ms	remaining: 542ms
136:	learn: 0.1436635	total: 634ms	remaining: 537ms
137:	learn: 0.1433116	total: 640ms	remaining: 533ms
138:	learn: 0.1422495	total: 645ms	remaining: 529ms
139:	learn: 0.1416034	total: 650ms	remaining: 525ms
140:	learn: 0.1409539	total: 655ms	remaining: 520ms
141:	learn: 0.1402196	total: 660ms	remaining: 516ms
142:	learn: 0.1396738	total: 664ms	remaining: 511ms
143:	learn: 0.1391573	total: 669ms	remaining: 507ms
144:	learn: 0.1382328	total: 674ms	remaining: 502ms
145:	learn: 

32:	learn: 0.2631219	total: 145ms	remaining: 969ms
33:	learn: 0.2602152	total: 150ms	remaining: 965ms
34:	learn: 0.2577131	total: 154ms	remaining: 961ms
35:	learn: 0.2553987	total: 159ms	remaining: 959ms
36:	learn: 0.2539818	total: 163ms	remaining: 954ms
37:	learn: 0.2515845	total: 168ms	remaining: 951ms
38:	learn: 0.2493232	total: 172ms	remaining: 946ms
39:	learn: 0.2472477	total: 177ms	remaining: 942ms
40:	learn: 0.2449942	total: 181ms	remaining: 938ms
41:	learn: 0.2425753	total: 186ms	remaining: 933ms
42:	learn: 0.2405567	total: 190ms	remaining: 929ms
43:	learn: 0.2393732	total: 195ms	remaining: 926ms
44:	learn: 0.2379194	total: 199ms	remaining: 922ms
45:	learn: 0.2369653	total: 204ms	remaining: 918ms
46:	learn: 0.2352047	total: 209ms	remaining: 915ms
47:	learn: 0.2333437	total: 213ms	remaining: 911ms
48:	learn: 0.2314808	total: 218ms	remaining: 906ms
49:	learn: 0.2293125	total: 222ms	remaining: 903ms
50:	learn: 0.2268121	total: 227ms	remaining: 900ms
51:	learn: 0.2254579	total: 232

231:	learn: 0.0960582	total: 1.13s	remaining: 102ms
232:	learn: 0.0954756	total: 1.13s	remaining: 97.3ms
233:	learn: 0.0952133	total: 1.14s	remaining: 92.4ms
234:	learn: 0.0946371	total: 1.14s	remaining: 87.5ms
235:	learn: 0.0940147	total: 1.15s	remaining: 82.6ms
236:	learn: 0.0936646	total: 1.15s	remaining: 77.7ms
237:	learn: 0.0930957	total: 1.16s	remaining: 72.9ms
238:	learn: 0.0928199	total: 1.16s	remaining: 68ms
239:	learn: 0.0924573	total: 1.17s	remaining: 63.1ms
240:	learn: 0.0917879	total: 1.17s	remaining: 58.2ms
241:	learn: 0.0914641	total: 1.17s	remaining: 53.4ms
242:	learn: 0.0911678	total: 1.18s	remaining: 48.5ms
243:	learn: 0.0908857	total: 1.18s	remaining: 43.7ms
244:	learn: 0.0906222	total: 1.19s	remaining: 38.8ms
245:	learn: 0.0903382	total: 1.19s	remaining: 34ms
246:	learn: 0.0899964	total: 1.2s	remaining: 29.1ms
247:	learn: 0.0896590	total: 1.2s	remaining: 24.3ms
248:	learn: 0.0891633	total: 1.21s	remaining: 19.4ms
249:	learn: 0.0887014	total: 1.21s	remaining: 14.6ms


173:	learn: 0.1249319	total: 824ms	remaining: 374ms
174:	learn: 0.1243509	total: 829ms	remaining: 370ms
175:	learn: 0.1238732	total: 834ms	remaining: 365ms
176:	learn: 0.1234090	total: 838ms	remaining: 360ms
177:	learn: 0.1225032	total: 843ms	remaining: 355ms
178:	learn: 0.1219422	total: 848ms	remaining: 351ms
179:	learn: 0.1214825	total: 853ms	remaining: 346ms
180:	learn: 0.1209834	total: 857ms	remaining: 341ms
181:	learn: 0.1205294	total: 862ms	remaining: 336ms
182:	learn: 0.1201051	total: 866ms	remaining: 331ms
183:	learn: 0.1194836	total: 871ms	remaining: 327ms
184:	learn: 0.1190926	total: 876ms	remaining: 322ms
185:	learn: 0.1187427	total: 881ms	remaining: 317ms
186:	learn: 0.1181415	total: 886ms	remaining: 313ms
187:	learn: 0.1176834	total: 890ms	remaining: 308ms
188:	learn: 0.1168671	total: 895ms	remaining: 303ms
189:	learn: 0.1164337	total: 900ms	remaining: 298ms
190:	learn: 0.1156437	total: 905ms	remaining: 294ms
191:	learn: 0.1149467	total: 909ms	remaining: 289ms
192:	learn: 

In [60]:
result.mean()

array([-0.21365901, -0.33755481, -0.20563054, -0.27438546, -0.28390637,
       -0.19753772, -0.21960883, -0.19822536, -0.16933637, -0.20692832])

In [ ]:
pred = gsExtC.predict_proba(test[feat])

In [ ]:
final = pd.DataFrame(pred)

In [ ]:
test = test_1.copy() 
test['sum_x_component_1'] = test['x_component_1']+test['x_component_2']+test['x_component_3']+test['x_component_4']+test['x_component_5']

In [ ]:
test = pd.concat([test,final],axis=1)

In [ ]:
def tweak(df):
    for i in range(len(df)):
        if df['sum_x_component_1'][i]>=1:
            df['class_1'][i]=1.0
            df['class_2'][i]=0.0
    return df

In [ ]:
test=test.rename(columns={0:'class_1',1:'class_2'})

In [ ]:
test.head()

In [ ]:
df = tweak(test)

In [ ]:
df.head()

In [ ]:
sub=df[['class_1','class_2']]

In [ ]:
sub=sub.rename(columns={'class_1':1,'class_2':2})

In [ ]:
sub.head()

In [ ]:
sub.to_excel('/kaggle/working/40.xlsx',index=False)